In [1]:
# @author Prerana Devadhar
# @date 09-27-21
# Revolution Simulator & Linear Regression Model

import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import random
import csv

class Revolution:
    i=None #instance variable for population size
    initCount=None #instance variable for initial proportion of revolutionaries amongst overall population
    count=None #instance variable for current proportion of revolutionaries amongst overall population
    countRev=None #instance variable for minimum threshold necessary for a revolution to succeed
    t=None #instance variable for individual revolutionary thresholds amongst overall population (list of size i)
    distType=None #instance variable for distribution type: 0-->normal, 1-->uniform, 2-->double normal
    distTypeList=None #instance variable storing distribution type as an array: [1,0,0]-->normal, [0,1,0]-->uniform, [0,0,1]-->double normal
    time=0 #instance variable for time (in rounds) needed for revolution to succeed or fail

    def __init__(self, countRev): #constructor; param: countRev (minimum threshold necessary for a revolution to succeed)
        self.distTypeList=[0, 0, 0] #initializes distTypeList to default values; distribution type will be specified later in constructor
        self.i=random.randrange(100, 1000, 2) #initializes i to a random even integer between range of 100 and 1000
        self.initCount=round(random.uniform(0, 0.3), 2) #initializes and rounds initCount to a random float between 0 and 0.3
        self.count=self.initCount #initializes count to the value of initCount
        self.countRev=countRev #initializes countRev to float parameter countRev
        self.distType=int(round(random.uniform(0, 2),0)) #initializes distType with a random integer elements between 0 and 2
        self.distTypeList[self.distType]=1 #specifies distribution type by changing element at index distType of distTypeList from default to 1
        if self.distType==0: #if distType==0-->normal distribution 
            self.t=np.random.normal(0.5, 0.12, self.i).tolist() #populates t with i random values from a normal distribution, with mu=0.5 and sigma=0.12
        elif self.distType==1: #else if distType==1-->uniform distribution 
            self.t=np.random.uniform(0, 2, self.i).tolist() #populates t with i random values from a unifrom distribution, with low=0 and high=2
        else: #else if distType==2-->double normal distribution 
            X1 = np.random.normal(0.25, 0.06, int(self.i/2)) #initializes X1 by populating it with i/2 random values from a normal distribution, with mu=0.25 and sigma=0.06
            X2 = np.random.normal(0.75, 0.06, int(self.i/2)) #initializes X1 by populating it with i/2 random values from a normal distribution, with mu=0.75 and sigma=0.06
            self.t=X1+X2 #concatenates X1 and X2 to initialize t
        self.t.sort() #sorts elements of t in their natural ascending order
    
    def revOccurrence(self): #function that determines if a revolution occurs with specified instance of Revolution
        while self.count<self.countRev: #loop that continues as long as minimum threshold necessary for a revolution to succeed has not been reached (or if a revolution succeeds or fails)
            self.time+=1; #incrementing time based on the round at which thresholds are checked (at round 1, time=1)
            countPerRound=0 #initializing variable that stores number of people who joined revolution in a given round
            for s in self.t: #loop that iterates through t (the list of individual thresholds amongst the population)
                if s<=self.count: #if an individual's threshold is less than or equal to current poportion of revolutionaries amongst the overall population
                    countPerRound+=1 #increment countPerRound to indicate that this person has joined the revolution
                    self.t.remove(s) #remove this person's individual threshold from the t to so that they no longer have to be checked in the next round
            if countPerRound==0: #if no one has joined the revolution in a given round
                return 0 # return 0 and close the loop to indicate that the revolution is stagnant and has failed
            self.count+=countPerRound/self.i #increment count by the proportion of new people that have joined the revolution amongst the overall population
        return 1 #if the condition of the while-loop is no longer satisfied, it means the revolution is successful, so return 1
    
    def revOccurrence2(self): #EXTRA function with behavioral change (created a new function so implementation is easier for group members) 
        while self.count<self.countRev:
            self.time+=1;
            countPerRound=0
            for s in self.t:
                if s<=self.count:
                    countPerRound+=1
                    self.t.remove(s)
            if countPerRound==0: 
                return 0
            self.count+=countPerRound/self.i
            for v in self.t: #iterate through the list of thresholds that have not yet been met
                v=v*(0.95**self.time)*(1-(self.count**3)) #multiple individual thresholds by a factor of (0.95*time^2)(1-count^3) so that it's a function of time and the current proportion of revolutionaries
        return 1
    
    @staticmethod
    def outputToCSV(): #function that creates instances of Revolution, calls revOccurrence(), and writes to CSV file
        with open('Revolutions.txt', mode='w') as revCSV: #opening CSV file of choice to begin writing
            revWriter=csv.writer(revCSV, delimiter=' ', quotechar='"', quoting=csv.QUOTE_MINIMAL) #initializing writer object
            revs=[] #list that will store output of revOccurrence when called on instances of Revolution
            revSuccesses=0 #local variable that stores total number of revolution instances that were successful
            for j in range(1000): #range of this loop specifies number of instances of Revolution to be created
                revX=Revolution(0.8) #creates new instance of Revolution with parameter being minimum threshold necessary for a revolution to succeed
                revs.append(revX.revOccurrence()) #calls revOccurrence() and appends output to revs to store success/failure of a revolution
                if revs[j]==1:
                    revSuccesses+=1 #increments revSuccesses by 1 if a revolution was successful
                revWriter.writerow([j, revX.i, revX.initCount, revX.countRev, 
                                    revX.distTypeList[0], revX.distTypeList[1], revX.distTypeList[2], 
                                    revX.time, round(revX.count,2), revs[j]]) #writes variables describing this instance of Revolution to new row of CSV file 
    @staticmethod
    def outputToCSV2(): #EXTRA function with behavioral change (created a new function so implementation is easier for group members)
        with open('Revolutions.txt', mode='w') as revCSV:
            revWriter=csv.writer(revCSV, delimiter=' ', quotechar='"', quoting=csv.QUOTE_MINIMAL)
            revs=[]
            revSuccesses=0
            for j in range(1000):
                revX=Revolution(0.8)
                revs.append(revX.revOccurrence2()) #calls corressponding EXTRA function with behavioral change
                if revs[j]==1:
                    revSuccesses+=1
                revWriter.writerow([j, revX.i, revX.initCount, revX.countRev, 
                                    revX.distTypeList[0], revX.distTypeList[1], revX.distTypeList[2], 
                                    revX.time, round(revX.count,2), revs[j]])
    
    @staticmethod
    def linRegression(): #function that runs linear regression on the data set contained within a specified CSV file
        rows=[] #local list variable that stores contents from rows of CSV file
        with open('Revolutions.txt', mode='r') as revCSV: #opening CSV file of choice to begin reading
            revReader=csv.reader(revCSV, delimiter=' ') #initializing reader object
            for row in revReader: 
                rows.append(row) #appending all rows of CSV file to rows list
        x=[] #local list variable that stores values of x-variables used in linear regression
        y=[] #local list variable that stores values of y-variables used in linear regression
        for k in range(1000):
            x.append([rows[k][1], rows[k][2], rows[k][4], rows[k][5], rows[k][6]]) #appends i, initCount, distTypeList[0], distTypeList[1], and distTypeList[2] for each instance of Revolution to x
            y.append(rows[k][7]) #appends proportion of revolutionaries amongst population at the time the revolution succeeds or fails (count) to y 
        x=np.array(x, dtype=float) #converts x from a list to an array so that it can be used for linear regression
        y=np.array(y, dtype=float) #converts y from a list to an array so that it can be used for linear regression
        model=LinearRegression().fit(x, y) #runs linear regression, which is stored in local variable model
        b=model.coef_ #initializes list b to store coefficients from linear regression
        a=model.intercept_ #initializes a to store intercept from linear regression
        rSquare=model.score(x, y) #initializes rSquare to R^2 from linear regression
        print("NEW LINEAR REGRESSION OUTPUT: ")
        print("coefficients(b): ", b)
        print("intercept(a): ", a)
        print("rSquare (r^2): ", rSquare)
        print(" ")
        
Revolution.outputToCSV() #runs outputToCSV(), which creates a data set and writes to CSV file ('Revolutions.txt')
Revolution.linRegression() #runs linear regression model on most recent contents of CSV file ('Revolutions.txt')
Revolution.outputToCSV2() #runs outputToCSV2(), which creates a data set using EXTRA features and writes to CSV file ('Revolutions.txt')
Revolution.linRegression() #runs linear regression model on most recent contents of CSV file ('Revolutions.txt') *note that this is the result of the EXTRA features being added

NEW LINEAR REGRESSION OUTPUT: 
coefficients(b):  [ 3.49567153e-03  2.61370830e+01 -1.86357371e+00  6.52785275e+00
 -4.66427904e+00]
intercept(a):  -0.04769924437560036
rSquare (r^2):  0.7398452295590414
 
NEW LINEAR REGRESSION OUTPUT: 
coefficients(b):  [ 4.07280753e-03  2.40565618e+01 -1.79649764e+00  6.24988261e+00
 -4.45338496e+00]
intercept(a):  -0.0072308499415161265
rSquare (r^2):  0.7585615596020785
 
